In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import naive_bayes
from sklearn.metrics import roc_auc_score
%matplotlib inline

In [2]:
df=pd.read_csv("5_6208501227823563186.csv")

In [3]:
df.head()

,Timestamp,Name,Gender,Age,State or Union Territories,Working Status,How many times you are going for shopping in a day?,What is/was your preference as a long-distance traveling option in this pandemic?,Should lockdown be introduced again?,What do you think about the reopening of Schools or Colleges?,How have you spent the lockdown at home?,"Vaccination or Economy, what should be the priority?",Your suggestions to the authority(Govt.),Target
0,11-14-2020 14:22:58,Soham Shah,Male,18-30,Maharashtra,Student,0-1,Happy at home,Yes,After the vaccine is available in market,Attending online/virtual lecture and courses,Vaccination,NaN,Positive
1,11-14-2020 14:35:35,Vivek Chandola,Male,18-30,Uttrakhand,Student,0-1,Personal Car,No,After the vaccine is available in market,"Learn new things like cooking, coding, gardeni...",Vaccination,NaN,Positive
2,11-14-2020 14:39:20,Rishav Ghosh,Male,18-30,West Bengal,Student,0-1,Happy at home,Yes,After the vaccine is available in market,NaN,Both....,NaN,Positive
3,11-14-2020 14:42:27,Saif khan,Male,18-30,Jharkhand,Student,0-1,Happy at home,Yes,After the vaccine is available in market,Yes I have spent my time with my family and pl...,From my side vaccination first if people are ...,All exams should be taken by online mode. This...,Positive
4,11-14-2020 14:49:47,Md kaisar,Male,18-30,Jharkhand,Student,0-1,Happy at home,Yes,After the vaccine is available in market,playing games 🎮,Vaccination,To increase awareness for protection against c...,Positive


In [6]:
df=df.drop(["Timestamp"],axis=1)

In [7]:
df.columns=['Name','Sex','Age','State','Working Status','Shopping','Travelling preference','Re-lockdown','scl-clg','Lockdown at home','Vaccination or Economy','Suggestions','Target']

In [8]:
df.describe().transpose()

,count,unique,top,freq
Name,213,203,Mehroz,5
Sex,219,3,Male,145
Age,213,4,18-30,194
State,218,25,West Bengal,81
Working Status,218,4,Student,168
Shopping,218,3,0-1,199
Travelling preference,219,4,Happy at home,106
Re-lockdown,216,2,No,111
scl-clg,219,4,After the vaccine is available in market,111
Lockdown at home,186,153,Yes,22


In [9]:
df.isnull().sum()/len(df)*100

Name                       4.910714
Sex                        2.232143
Age                        4.910714
State                      2.678571
Working Status             2.678571
Shopping                   2.678571
Travelling preference      2.232143
Re-lockdown                3.571429
scl-clg                    2.232143
Lockdown at home          16.964286
Vaccination or Economy    12.500000
Suggestions               36.160714
Target                     0.000000
dtype: float64

In [10]:
#filling null values
df["Suggestions"]=df["Suggestions"].fillna('No Suggestions')
df["Vaccination or Economy"]=df["Vaccination or Economy"].fillna(method='ffill')
df["Lockdown at home"]=df["Lockdown at home"].fillna("No response")
df["Name"]=df["Name"].fillna(method='ffill')
df["scl-clg"]=df["scl-clg"].fillna(method='ffill')
df["Re-lockdown"]=df["Re-lockdown"].fillna(method='ffill')
df["Travelling preference"]=df["Travelling preference"].fillna(method='ffill')
df["Shopping"]=df["Shopping"].fillna(method='ffill')
df["Working Status"]=df["Working Status"].fillna(method='ffill')
df["State"]=df["State"].fillna(method='ffill')
df["Age"]=df["Age"].fillna(method='ffill')
df["Sex"]=df["Sex"].fillna(method='ffill')
df["Target"]=df["Target"].fillna("Positive")

In [11]:
df.isnull().sum()/len(df)*100

Name                      0.0
Sex                       0.0
Age                       0.0
State                     0.0
Working Status            0.0
Shopping                  0.0
Travelling preference     0.0
Re-lockdown               0.0
scl-clg                   0.0
Lockdown at home          0.0
Vaccination or Economy    0.0
Suggestions               0.0
Target                    0.0
dtype: float64

In [12]:
df["Review"]=df["Travelling preference"]+' '+df["Re-lockdown"]+" "+df["scl-clg"]+" "+df["Lockdown at home"]+" "+df["Vaccination or Economy"]+" "+df["Suggestions"]

In [13]:
df.dtypes

Name                      object
Sex                       object
Age                       object
State                     object
Working Status            object
Shopping                  object
Travelling preference     object
Re-lockdown               object
scl-clg                   object
Lockdown at home          object
Vaccination or Economy    object
Suggestions               object
Target                    object
Review                    object
dtype: object

In [14]:
df.head()

,Name,Sex,Age,State,Working Status,Shopping,Travelling preference,Re-lockdown,scl-clg,Lockdown at home,Vaccination or Economy,Suggestions,Target,Review
0,Soham Shah,Male,18-30,Maharashtra,Student,0-1,Happy at home,Yes,After the vaccine is available in market,Attending online/virtual lecture and courses,Vaccination,No Suggestions,Positive,Happy at home Yes After the vaccine is availab...
1,Vivek Chandola,Male,18-30,Uttrakhand,Student,0-1,Personal Car,No,After the vaccine is available in market,"Learn new things like cooking, coding, gardeni...",Vaccination,No Suggestions,Positive,Personal Car No After the vaccine is available...
2,Rishav Ghosh,Male,18-30,West Bengal,Student,0-1,Happy at home,Yes,After the vaccine is available in market,No response,Both....,No Suggestions,Positive,Happy at home Yes After the vaccine is availab...
3,Saif khan,Male,18-30,Jharkhand,Student,0-1,Happy at home,Yes,After the vaccine is available in market,Yes I have spent my time with my family and pl...,From my side vaccination first if people are ...,All exams should be taken by online mode. This...,Positive,Happy at home Yes After the vaccine is availab...
4,Md kaisar,Male,18-30,Jharkhand,Student,0-1,Happy at home,Yes,After the vaccine is available in market,playing games 🎮,Vaccination,To increase awareness for protection against c...,Positive,Happy at home Yes After the vaccine is availab...


In [15]:
cols=df.columns
df[["Target","Review"]]=df[["Review","Target"]]

In [16]:
df.columns=['Name','Sex','Age','State','Working Status','Shopping','Travelling preference','Re-lockdown','scl-clg','Lockdown at home','Vaccination or Economy','Suggestions','Review',"Target"]

In [17]:
df.head()

,Name,Sex,Age,State,Working Status,Shopping,Travelling preference,Re-lockdown,scl-clg,Lockdown at home,Vaccination or Economy,Suggestions,Review,Target
0,Soham Shah,Male,18-30,Maharashtra,Student,0-1,Happy at home,Yes,After the vaccine is available in market,Attending online/virtual lecture and courses,Vaccination,No Suggestions,Happy at home Yes After the vaccine is availab...,Positive
1,Vivek Chandola,Male,18-30,Uttrakhand,Student,0-1,Personal Car,No,After the vaccine is available in market,"Learn new things like cooking, coding, gardeni...",Vaccination,No Suggestions,Personal Car No After the vaccine is available...,Positive
2,Rishav Ghosh,Male,18-30,West Bengal,Student,0-1,Happy at home,Yes,After the vaccine is available in market,No response,Both....,No Suggestions,Happy at home Yes After the vaccine is availab...,Positive
3,Saif khan,Male,18-30,Jharkhand,Student,0-1,Happy at home,Yes,After the vaccine is available in market,Yes I have spent my time with my family and pl...,From my side vaccination first if people are ...,All exams should be taken by online mode. This...,Happy at home Yes After the vaccine is availab...,Positive
4,Md kaisar,Male,18-30,Jharkhand,Student,0-1,Happy at home,Yes,After the vaccine is available in market,playing games 🎮,Vaccination,To increase awareness for protection against c...,Happy at home Yes After the vaccine is availab...,Positive


NLP

In [18]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from nltk.tokenize import RegexpTokenizer
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords

In [19]:
tokenizer=RegexpTokenizer(r'\w+')
ps=PorterStemmer()
eng_stopwords=set(stopwords.words("English"))

In [20]:
def stemmed(review):
    review=review.lower()
#     review=review.replace("<br /><br />"," ")
    tokeized=tokenizer.tokenize(review)
    stopwords_removed=[w for w in tokeized if w not in eng_stopwords]
    stemmed_review=[ps.stem(w) for w in stopwords_removed] 
    final=' '.join(stemmed_review)
    return final

In [21]:
data_x=df.iloc[:,12]
data_y=df.iloc[:,13]
print(data_x.shape,data_y.shape)

(224,) (224,)


In [ ]:
data_x

In [22]:
data_y

0      Positive
1      Positive
2      Positive
3      Positive
4      Positive
         ...   
219    Positive
220    Negative
221    Positive
222    Negative
223    Positive
Name: Target, Length: 224, dtype: object

In [23]:
x_data=[stemmed(i) for i in data_x]

In [25]:
from sklearn.feature_extraction.text import CountVectorizer

In [26]:
cv=CountVectorizer(ngram_range=(1,2))

In [27]:
x=cv.fit_transform(x_data)
# x_test=cv.transform(test_cleaned)

In [28]:
print(len(cv.get_feature_names()))

2815


In [29]:
lb=LabelEncoder()
data_y=lb.fit_transform(data_y)

In [30]:
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

In [32]:
#Catboost classifier
clf=CatBoostClassifier()
clf.fit(x,data_y)

Learning rate set to 0.005439
0:	learn: 0.6905900	total: 156ms	remaining: 2m 35s
1:	learn: 0.6877161	total: 190ms	remaining: 1m 34s
2:	learn: 0.6854140	total: 221ms	remaining: 1m 13s
3:	learn: 0.6828774	total: 252ms	remaining: 1m 2s
4:	learn: 0.6803579	total: 283ms	remaining: 56.3s
5:	learn: 0.6776112	total: 310ms	remaining: 51.4s
6:	learn: 0.6756498	total: 337ms	remaining: 47.8s
7:	learn: 0.6741014	total: 359ms	remaining: 44.6s
8:	learn: 0.6714505	total: 393ms	remaining: 43.2s
9:	learn: 0.6690739	total: 426ms	remaining: 42.2s
10:	learn: 0.6667334	total: 461ms	remaining: 41.5s
11:	learn: 0.6640663	total: 493ms	remaining: 40.6s
12:	learn: 0.6614892	total: 525ms	remaining: 39.8s
13:	learn: 0.6592127	total: 557ms	remaining: 39.2s
14:	learn: 0.6570090	total: 590ms	remaining: 38.7s
15:	learn: 0.6550536	total: 627ms	remaining: 38.5s
16:	learn: 0.6528665	total: 661ms	remaining: 38.2s
17:	learn: 0.6508049	total: 693ms	remaining: 37.8s
18:	learn: 0.6491276	total: 725ms	remaining: 37.4s
19:	lear

163:	learn: 0.4844116	total: 5.38s	remaining: 27.5s
164:	learn: 0.4838209	total: 5.42s	remaining: 27.4s
165:	learn: 0.4834462	total: 5.46s	remaining: 27.4s
166:	learn: 0.4829831	total: 5.49s	remaining: 27.4s
167:	learn: 0.4821387	total: 5.53s	remaining: 27.4s
168:	learn: 0.4810256	total: 5.57s	remaining: 27.4s
169:	learn: 0.4805087	total: 5.6s	remaining: 27.4s
170:	learn: 0.4798313	total: 5.64s	remaining: 27.3s
171:	learn: 0.4794130	total: 5.67s	remaining: 27.3s
172:	learn: 0.4788511	total: 5.71s	remaining: 27.3s
173:	learn: 0.4781102	total: 5.74s	remaining: 27.3s
174:	learn: 0.4767427	total: 5.77s	remaining: 27.2s
175:	learn: 0.4757352	total: 5.81s	remaining: 27.2s
176:	learn: 0.4752369	total: 5.84s	remaining: 27.2s
177:	learn: 0.4747726	total: 5.88s	remaining: 27.2s
178:	learn: 0.4733646	total: 5.92s	remaining: 27.1s
179:	learn: 0.4728449	total: 5.95s	remaining: 27.1s
180:	learn: 0.4719084	total: 5.99s	remaining: 27.1s
181:	learn: 0.4711218	total: 6.02s	remaining: 27.1s
182:	learn: 0

328:	learn: 0.4045157	total: 10.4s	remaining: 21.1s
329:	learn: 0.4041325	total: 10.4s	remaining: 21.1s
330:	learn: 0.4038573	total: 10.4s	remaining: 21.1s
331:	learn: 0.4034777	total: 10.4s	remaining: 21s
332:	learn: 0.4033160	total: 10.5s	remaining: 21s
333:	learn: 0.4029592	total: 10.5s	remaining: 20.9s
334:	learn: 0.4026025	total: 10.5s	remaining: 20.9s
335:	learn: 0.4023478	total: 10.6s	remaining: 20.9s
336:	learn: 0.4021484	total: 10.6s	remaining: 20.9s
337:	learn: 0.4019443	total: 10.6s	remaining: 20.8s
338:	learn: 0.4014167	total: 10.7s	remaining: 20.8s
339:	learn: 0.4011713	total: 10.7s	remaining: 20.8s
340:	learn: 0.4010077	total: 10.7s	remaining: 20.8s
341:	learn: 0.4004947	total: 10.8s	remaining: 20.7s
342:	learn: 0.4002979	total: 10.8s	remaining: 20.7s
343:	learn: 0.4000697	total: 10.8s	remaining: 20.7s
344:	learn: 0.3996393	total: 10.9s	remaining: 20.6s
345:	learn: 0.3992292	total: 10.9s	remaining: 20.6s
346:	learn: 0.3991969	total: 10.9s	remaining: 20.5s
347:	learn: 0.39

487:	learn: 0.3642269	total: 15.4s	remaining: 16.1s
488:	learn: 0.3640144	total: 15.4s	remaining: 16.1s
489:	learn: 0.3639644	total: 15.4s	remaining: 16s
490:	learn: 0.3638452	total: 15.4s	remaining: 16s
491:	learn: 0.3636317	total: 15.4s	remaining: 15.9s
492:	learn: 0.3633778	total: 15.5s	remaining: 15.9s
493:	learn: 0.3631917	total: 15.5s	remaining: 15.9s
494:	learn: 0.3630913	total: 15.5s	remaining: 15.8s
495:	learn: 0.3630313	total: 15.6s	remaining: 15.8s
496:	learn: 0.3628177	total: 15.6s	remaining: 15.8s
497:	learn: 0.3626164	total: 15.6s	remaining: 15.7s
498:	learn: 0.3625054	total: 15.7s	remaining: 15.7s
499:	learn: 0.3623197	total: 15.7s	remaining: 15.7s
500:	learn: 0.3622484	total: 15.7s	remaining: 15.7s
501:	learn: 0.3618005	total: 15.8s	remaining: 15.6s
502:	learn: 0.3616842	total: 15.8s	remaining: 15.6s
503:	learn: 0.3609824	total: 15.8s	remaining: 15.6s
504:	learn: 0.3608655	total: 15.8s	remaining: 15.5s
505:	learn: 0.3607575	total: 15.9s	remaining: 15.5s
506:	learn: 0.36

647:	learn: 0.3359604	total: 20.3s	remaining: 11s
648:	learn: 0.3358804	total: 20.3s	remaining: 11s
649:	learn: 0.3358172	total: 20.3s	remaining: 11s
650:	learn: 0.3356034	total: 20.4s	remaining: 10.9s
651:	learn: 0.3353734	total: 20.4s	remaining: 10.9s
652:	learn: 0.3352787	total: 20.4s	remaining: 10.9s
653:	learn: 0.3352207	total: 20.5s	remaining: 10.8s
654:	learn: 0.3351400	total: 20.5s	remaining: 10.8s
655:	learn: 0.3349874	total: 20.5s	remaining: 10.8s
656:	learn: 0.3346409	total: 20.5s	remaining: 10.7s
657:	learn: 0.3343767	total: 20.6s	remaining: 10.7s
658:	learn: 0.3343425	total: 20.6s	remaining: 10.6s
659:	learn: 0.3341889	total: 20.6s	remaining: 10.6s
660:	learn: 0.3336602	total: 20.6s	remaining: 10.6s
661:	learn: 0.3335155	total: 20.7s	remaining: 10.6s
662:	learn: 0.3334327	total: 20.7s	remaining: 10.5s
663:	learn: 0.3333414	total: 20.7s	remaining: 10.5s
664:	learn: 0.3332386	total: 20.8s	remaining: 10.5s
665:	learn: 0.3331762	total: 20.8s	remaining: 10.4s
666:	learn: 0.3330

806:	learn: 0.3116243	total: 25.2s	remaining: 6.03s
807:	learn: 0.3115038	total: 25.3s	remaining: 6s
808:	learn: 0.3114317	total: 25.3s	remaining: 5.97s
809:	learn: 0.3112233	total: 25.3s	remaining: 5.94s
810:	learn: 0.3111227	total: 25.4s	remaining: 5.91s
811:	learn: 0.3110463	total: 25.4s	remaining: 5.88s
812:	learn: 0.3109739	total: 25.4s	remaining: 5.85s
813:	learn: 0.3108744	total: 25.5s	remaining: 5.82s
814:	learn: 0.3107492	total: 25.5s	remaining: 5.79s
815:	learn: 0.3106104	total: 25.5s	remaining: 5.76s
816:	learn: 0.3105705	total: 25.6s	remaining: 5.72s
817:	learn: 0.3105205	total: 25.6s	remaining: 5.69s
818:	learn: 0.3104502	total: 25.6s	remaining: 5.66s
819:	learn: 0.3103409	total: 25.6s	remaining: 5.63s
820:	learn: 0.3102606	total: 25.7s	remaining: 5.6s
821:	learn: 0.3101040	total: 25.7s	remaining: 5.57s
822:	learn: 0.3099085	total: 25.7s	remaining: 5.53s
823:	learn: 0.3098275	total: 25.8s	remaining: 5.5s
824:	learn: 0.3096231	total: 25.8s	remaining: 5.47s
825:	learn: 0.309

965:	learn: 0.2867814	total: 30.1s	remaining: 1.06s
966:	learn: 0.2866707	total: 30.2s	remaining: 1.03s
967:	learn: 0.2862870	total: 30.2s	remaining: 998ms
968:	learn: 0.2862288	total: 30.2s	remaining: 967ms
969:	learn: 0.2861582	total: 30.3s	remaining: 936ms
970:	learn: 0.2858687	total: 30.3s	remaining: 905ms
971:	learn: 0.2858073	total: 30.3s	remaining: 874ms
972:	learn: 0.2853837	total: 30.4s	remaining: 842ms
973:	learn: 0.2853134	total: 30.4s	remaining: 811ms
974:	learn: 0.2852259	total: 30.4s	remaining: 780ms
975:	learn: 0.2848958	total: 30.4s	remaining: 749ms
976:	learn: 0.2848162	total: 30.5s	remaining: 717ms
977:	learn: 0.2847354	total: 30.5s	remaining: 686ms
978:	learn: 0.2846644	total: 30.5s	remaining: 655ms
979:	learn: 0.2844781	total: 30.6s	remaining: 624ms
980:	learn: 0.2844188	total: 30.6s	remaining: 593ms
981:	learn: 0.2839088	total: 30.6s	remaining: 561ms
982:	learn: 0.2838298	total: 30.7s	remaining: 530ms
983:	learn: 0.2837679	total: 30.7s	remaining: 499ms
984:	learn: 

In [34]:
from sklearn.model_selection import cross_validate,cross_val_score
from sklearn.metrics import f1_score,make_scorer

In [35]:
#calculating f1 score for catboost 
def f1(y_true, y_pred):
    return f1_score(y_true, y_pred,average='micro')

f1_s = make_scorer(f1, greater_is_better=True)
scores= cross_val_score(clf, x, data_y, cv = 5, scoring = f1_s)
scores.mean()

Learning rate set to 0.004942
0:	learn: 0.6912481	total: 24.7ms	remaining: 24.6s
1:	learn: 0.6887677	total: 54.3ms	remaining: 27.1s
2:	learn: 0.6871199	total: 81.4ms	remaining: 27s
3:	learn: 0.6856845	total: 109ms	remaining: 27.1s
4:	learn: 0.6834830	total: 137ms	remaining: 27.2s
5:	learn: 0.6819300	total: 164ms	remaining: 27.2s
6:	learn: 0.6800777	total: 192ms	remaining: 27.2s
7:	learn: 0.6783372	total: 204ms	remaining: 25.3s
8:	learn: 0.6756782	total: 227ms	remaining: 25s
9:	learn: 0.6737639	total: 247ms	remaining: 24.5s
10:	learn: 0.6715476	total: 270ms	remaining: 24.3s
11:	learn: 0.6698630	total: 295ms	remaining: 24.3s
12:	learn: 0.6676284	total: 320ms	remaining: 24.3s
13:	learn: 0.6657203	total: 352ms	remaining: 24.8s
14:	learn: 0.6632948	total: 384ms	remaining: 25.2s
15:	learn: 0.6612811	total: 415ms	remaining: 25.5s
16:	learn: 0.6593668	total: 447ms	remaining: 25.9s
17:	learn: 0.6577730	total: 478ms	remaining: 26.1s
18:	learn: 0.6556555	total: 508ms	remaining: 26.2s
19:	learn: 0

163:	learn: 0.4985547	total: 4.93s	remaining: 25.1s
164:	learn: 0.4983792	total: 4.96s	remaining: 25.1s
165:	learn: 0.4974868	total: 4.99s	remaining: 25.1s
166:	learn: 0.4970362	total: 5.02s	remaining: 25s
167:	learn: 0.4962337	total: 5.05s	remaining: 25s
168:	learn: 0.4961117	total: 5.09s	remaining: 25s
169:	learn: 0.4954061	total: 5.12s	remaining: 25s
170:	learn: 0.4950481	total: 5.15s	remaining: 25s
171:	learn: 0.4945283	total: 5.18s	remaining: 24.9s
172:	learn: 0.4938075	total: 5.21s	remaining: 24.9s
173:	learn: 0.4932443	total: 5.24s	remaining: 24.9s
174:	learn: 0.4927020	total: 5.27s	remaining: 24.8s
175:	learn: 0.4922086	total: 5.3s	remaining: 24.8s
176:	learn: 0.4916208	total: 5.33s	remaining: 24.8s
177:	learn: 0.4911696	total: 5.37s	remaining: 24.8s
178:	learn: 0.4909119	total: 5.4s	remaining: 24.8s
179:	learn: 0.4900809	total: 5.43s	remaining: 24.7s
180:	learn: 0.4896226	total: 5.46s	remaining: 24.7s
181:	learn: 0.4892252	total: 5.49s	remaining: 24.7s
182:	learn: 0.4884427	to

325:	learn: 0.4248476	total: 9.89s	remaining: 20.5s
326:	learn: 0.4244746	total: 9.93s	remaining: 20.4s
327:	learn: 0.4242072	total: 9.96s	remaining: 20.4s
328:	learn: 0.4240601	total: 9.99s	remaining: 20.4s
329:	learn: 0.4234692	total: 10s	remaining: 20.3s
330:	learn: 0.4231030	total: 10.1s	remaining: 20.3s
331:	learn: 0.4228130	total: 10.1s	remaining: 20.3s
332:	learn: 0.4225914	total: 10.1s	remaining: 20.3s
333:	learn: 0.4222747	total: 10.2s	remaining: 20.2s
334:	learn: 0.4218362	total: 10.2s	remaining: 20.2s
335:	learn: 0.4215001	total: 10.2s	remaining: 20.2s
336:	learn: 0.4213242	total: 10.3s	remaining: 20.2s
337:	learn: 0.4211926	total: 10.3s	remaining: 20.2s
338:	learn: 0.4208022	total: 10.3s	remaining: 20.1s
339:	learn: 0.4204310	total: 10.4s	remaining: 20.1s
340:	learn: 0.4202150	total: 10.4s	remaining: 20.1s
341:	learn: 0.4200856	total: 10.4s	remaining: 20s
342:	learn: 0.4198082	total: 10.4s	remaining: 20s
343:	learn: 0.4197426	total: 10.5s	remaining: 20s
344:	learn: 0.419585

488:	learn: 0.3814356	total: 15s	remaining: 15.7s
489:	learn: 0.3812551	total: 15.1s	remaining: 15.7s
490:	learn: 0.3809773	total: 15.1s	remaining: 15.6s
491:	learn: 0.3809132	total: 15.1s	remaining: 15.6s
492:	learn: 0.3808081	total: 15.1s	remaining: 15.6s
493:	learn: 0.3806942	total: 15.2s	remaining: 15.5s
494:	learn: 0.3806391	total: 15.2s	remaining: 15.5s
495:	learn: 0.3805107	total: 15.2s	remaining: 15.5s
496:	learn: 0.3803899	total: 15.3s	remaining: 15.4s
497:	learn: 0.3802797	total: 15.3s	remaining: 15.4s
498:	learn: 0.3800171	total: 15.3s	remaining: 15.4s
499:	learn: 0.3797076	total: 15.3s	remaining: 15.3s
500:	learn: 0.3795915	total: 15.4s	remaining: 15.3s
501:	learn: 0.3793524	total: 15.4s	remaining: 15.3s
502:	learn: 0.3792054	total: 15.4s	remaining: 15.2s
503:	learn: 0.3790748	total: 15.4s	remaining: 15.2s
504:	learn: 0.3789397	total: 15.5s	remaining: 15.2s
505:	learn: 0.3787317	total: 15.5s	remaining: 15.1s
506:	learn: 0.3784878	total: 15.5s	remaining: 15.1s
507:	learn: 0.

649:	learn: 0.3513415	total: 19.7s	remaining: 10.6s
650:	learn: 0.3510957	total: 19.8s	remaining: 10.6s
651:	learn: 0.3507846	total: 19.8s	remaining: 10.6s
652:	learn: 0.3506843	total: 19.8s	remaining: 10.5s
653:	learn: 0.3503441	total: 19.9s	remaining: 10.5s
654:	learn: 0.3502563	total: 19.9s	remaining: 10.5s
655:	learn: 0.3500804	total: 19.9s	remaining: 10.4s
656:	learn: 0.3499757	total: 20s	remaining: 10.4s
657:	learn: 0.3498249	total: 20s	remaining: 10.4s
658:	learn: 0.3497274	total: 20s	remaining: 10.4s
659:	learn: 0.3496248	total: 20.1s	remaining: 10.3s
660:	learn: 0.3495394	total: 20.1s	remaining: 10.3s
661:	learn: 0.3494258	total: 20.1s	remaining: 10.3s
662:	learn: 0.3489885	total: 20.1s	remaining: 10.2s
663:	learn: 0.3488153	total: 20.2s	remaining: 10.2s
664:	learn: 0.3486787	total: 20.2s	remaining: 10.2s
665:	learn: 0.3484296	total: 20.2s	remaining: 10.1s
666:	learn: 0.3481684	total: 20.3s	remaining: 10.1s
667:	learn: 0.3476428	total: 20.3s	remaining: 10.1s
668:	learn: 0.3475

...

364:	learn: 0.4384166	total: 9.96s	remaining: 17.3s
365:	learn: 0.4381737	total: 9.99s	remaining: 17.3s
366:	learn: 0.4377983	total: 10s	remaining: 17.3s
367:	learn: 0.4373791	total: 10.1s	remaining: 17.3s
368:	learn: 0.4371519	total: 10.1s	remaining: 17.2s
369:	learn: 0.4367717	total: 10.1s	remaining: 17.2s
370:	learn: 0.4366813	total: 10.1s	remaining: 17.2s
371:	learn: 0.4359072	total: 10.2s	remaining: 17.2s
372:	learn: 0.4356727	total: 10.2s	remaining: 17.1s
373:	learn: 0.4350687	total: 10.2s	remaining: 17.1s
374:	learn: 0.4346588	total: 10.3s	remaining: 17.1s
375:	learn: 0.4341674	total: 10.3s	remaining: 17.1s
376:	learn: 0.4339534	total: 10.3s	remaining: 17s
377:	learn: 0.4336494	total: 10.3s	remaining: 17s
378:	learn: 0.4334869	total: 10.4s	remaining: 17s
379:	learn: 0.4332693	total: 10.4s	remaining: 17s
380:	learn: 0.4329437	total: 10.4s	remaining: 16.9s
381:	learn: 0.4327159	total: 10.5s	remaining: 16.9s
382:	learn: 0.4324973	total: 10.5s	remaining: 16.9s
383:	learn: 0.4319792	

528:	learn: 0.3969369	total: 14.7s	remaining: 13.1s
529:	learn: 0.3965294	total: 14.8s	remaining: 13.1s
530:	learn: 0.3964303	total: 14.8s	remaining: 13.1s
531:	learn: 0.3962662	total: 14.8s	remaining: 13.1s
532:	learn: 0.3961169	total: 14.9s	remaining: 13s
533:	learn: 0.3960231	total: 14.9s	remaining: 13s
534:	learn: 0.3958935	total: 15s	remaining: 13s
535:	learn: 0.3958526	total: 15s	remaining: 13s
536:	learn: 0.3957015	total: 15s	remaining: 13s
537:	learn: 0.3956027	total: 15.1s	remaining: 12.9s
538:	learn: 0.3953879	total: 15.1s	remaining: 12.9s
539:	learn: 0.3953136	total: 15.1s	remaining: 12.9s
540:	learn: 0.3952185	total: 15.1s	remaining: 12.8s
541:	learn: 0.3951020	total: 15.2s	remaining: 12.8s
542:	learn: 0.3949755	total: 15.2s	remaining: 12.8s
543:	learn: 0.3948673	total: 15.2s	remaining: 12.8s
544:	learn: 0.3947866	total: 15.3s	remaining: 12.7s
545:	learn: 0.3944012	total: 15.3s	remaining: 12.7s
546:	learn: 0.3941719	total: 15.3s	remaining: 12.7s
547:	learn: 0.3939737	total:

688:	learn: 0.3690484	total: 19.3s	remaining: 8.73s
689:	learn: 0.3687881	total: 19.4s	remaining: 8.71s
690:	learn: 0.3686075	total: 19.4s	remaining: 8.68s
691:	learn: 0.3683139	total: 19.4s	remaining: 8.65s
692:	learn: 0.3681461	total: 19.5s	remaining: 8.62s
693:	learn: 0.3680384	total: 19.5s	remaining: 8.59s
694:	learn: 0.3679880	total: 19.5s	remaining: 8.56s
695:	learn: 0.3678992	total: 19.5s	remaining: 8.54s
696:	learn: 0.3677055	total: 19.6s	remaining: 8.51s
697:	learn: 0.3676183	total: 19.6s	remaining: 8.48s
698:	learn: 0.3673447	total: 19.6s	remaining: 8.45s
699:	learn: 0.3672083	total: 19.7s	remaining: 8.43s
700:	learn: 0.3669683	total: 19.7s	remaining: 8.4s
701:	learn: 0.3669232	total: 19.7s	remaining: 8.37s
702:	learn: 0.3664987	total: 19.7s	remaining: 8.34s
703:	learn: 0.3663824	total: 19.8s	remaining: 8.31s
704:	learn: 0.3662604	total: 19.8s	remaining: 8.28s
705:	learn: 0.3658286	total: 19.8s	remaining: 8.25s
706:	learn: 0.3656870	total: 19.8s	remaining: 8.22s
707:	learn: 0

850:	learn: 0.3412577	total: 23.6s	remaining: 4.14s
851:	learn: 0.3411288	total: 23.7s	remaining: 4.11s
852:	learn: 0.3410549	total: 23.7s	remaining: 4.08s
853:	learn: 0.3408301	total: 23.7s	remaining: 4.06s
854:	learn: 0.3407288	total: 23.8s	remaining: 4.03s
855:	learn: 0.3405788	total: 23.8s	remaining: 4s
856:	learn: 0.3404838	total: 23.8s	remaining: 3.97s
857:	learn: 0.3403631	total: 23.8s	remaining: 3.95s
858:	learn: 0.3402893	total: 23.9s	remaining: 3.92s
859:	learn: 0.3401281	total: 23.9s	remaining: 3.89s
860:	learn: 0.3400492	total: 23.9s	remaining: 3.86s
861:	learn: 0.3398671	total: 24s	remaining: 3.84s
862:	learn: 0.3397420	total: 24s	remaining: 3.81s
863:	learn: 0.3395915	total: 24s	remaining: 3.78s
864:	learn: 0.3392656	total: 24.1s	remaining: 3.75s
865:	learn: 0.3386106	total: 24.1s	remaining: 3.73s
866:	learn: 0.3383048	total: 24.1s	remaining: 3.7s
867:	learn: 0.3381954	total: 24.1s	remaining: 3.67s
868:	learn: 0.3381047	total: 24.2s	remaining: 3.64s
869:	learn: 0.3380362	

0.7951515151515152

In [36]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RepeatedKFold
from sklearn.ensemble import StackingClassifier
from sklearn.model_selection import cross_validate,cross_val_score

In [146]:
# Xg=XGBClassifier(gamma=3.021935499258692,max_depth=int(8.172425201187867),n_estimators=int(128.2382303581367),subsample=0.8463044334965718)
# Lgbm=LGBMClassifier(n_estimators=100,learning_rate=0.1,random_state=42,num_leaves=200)
    
# Cataboost=CatBoostClassifier(depth=int(6.298),n_estimators=int(448.8))

# stacking naive bayes,knn,logistic regression,svm
estimators = [('mn', mn),('lr',lr),('sv',sv),('Xg',Xg)]

model = StackingClassifier(estimators=estimators)

In [147]:
# Best fit 
scores2= cross_val_score(model, x, data_y, cv = 5, scoring = 'accuracy')
scores2.mean()

0.8084848484848484

In [151]:
#accuracy of catboost
scores3= cross_val_score(Cataboost, x, data_y, cv = 5, scoring = 'accuracy')
scores3.mean()

Learning rate set to 0.142038
0:	learn: 1.0232992	total: 295ms	remaining: 2m 11s
1:	learn: 0.9675260	total: 320ms	remaining: 1m 11s
2:	learn: 0.9212198	total: 336ms	remaining: 49.8s
3:	learn: 0.8856003	total: 347ms	remaining: 38.5s
4:	learn: 0.8582063	total: 358ms	remaining: 31.7s
5:	learn: 0.8255678	total: 369ms	remaining: 27.1s
6:	learn: 0.8084529	total: 379ms	remaining: 23.9s
7:	learn: 0.7842782	total: 390ms	remaining: 21.5s
8:	learn: 0.7661669	total: 401ms	remaining: 19.6s
9:	learn: 0.7459637	total: 412ms	remaining: 18s
10:	learn: 0.7409948	total: 415ms	remaining: 16.5s
11:	learn: 0.7340245	total: 426ms	remaining: 15.5s
12:	learn: 0.7178055	total: 438ms	remaining: 14.7s
13:	learn: 0.7132163	total: 448ms	remaining: 13.9s
14:	learn: 0.7113296	total: 459ms	remaining: 13.2s
15:	learn: 0.7088525	total: 471ms	remaining: 12.7s
16:	learn: 0.7048212	total: 482ms	remaining: 12.2s
17:	learn: 0.6943520	total: 495ms	remaining: 11.8s
18:	learn: 0.6805944	total: 507ms	remaining: 11.4s
19:	learn: 

162:	learn: 0.2557009	total: 2.12s	remaining: 3.7s
163:	learn: 0.2548617	total: 2.13s	remaining: 3.69s
164:	learn: 0.2540290	total: 2.14s	remaining: 3.67s
165:	learn: 0.2531753	total: 2.15s	remaining: 3.66s
166:	learn: 0.2522118	total: 2.16s	remaining: 3.64s
167:	learn: 0.2493318	total: 2.17s	remaining: 3.62s
168:	learn: 0.2486253	total: 2.19s	remaining: 3.61s
169:	learn: 0.2477661	total: 2.19s	remaining: 3.59s
170:	learn: 0.2463959	total: 2.21s	remaining: 3.57s
171:	learn: 0.2456240	total: 2.22s	remaining: 3.56s
172:	learn: 0.2447134	total: 2.23s	remaining: 3.54s
173:	learn: 0.2441023	total: 2.24s	remaining: 3.52s
174:	learn: 0.2434483	total: 2.25s	remaining: 3.5s
175:	learn: 0.2422853	total: 2.26s	remaining: 3.49s
176:	learn: 0.2414728	total: 2.27s	remaining: 3.47s
177:	learn: 0.2401752	total: 2.28s	remaining: 3.46s
178:	learn: 0.2391762	total: 2.29s	remaining: 3.44s
179:	learn: 0.2384006	total: 2.3s	remaining: 3.43s
180:	learn: 0.2375349	total: 2.31s	remaining: 3.42s
181:	learn: 0.2

332:	learn: 0.1371120	total: 4.11s	remaining: 1.42s
333:	learn: 0.1368505	total: 4.12s	remaining: 1.41s
334:	learn: 0.1366183	total: 4.13s	remaining: 1.39s
335:	learn: 0.1363330	total: 4.15s	remaining: 1.38s
336:	learn: 0.1360610	total: 4.16s	remaining: 1.37s
337:	learn: 0.1358214	total: 4.17s	remaining: 1.36s
338:	learn: 0.1352675	total: 4.18s	remaining: 1.34s
339:	learn: 0.1350705	total: 4.2s	remaining: 1.33s
340:	learn: 0.1335092	total: 4.21s	remaining: 1.32s
341:	learn: 0.1332086	total: 4.22s	remaining: 1.31s
342:	learn: 0.1329173	total: 4.24s	remaining: 1.3s
343:	learn: 0.1325329	total: 4.25s	remaining: 1.28s
344:	learn: 0.1322964	total: 4.26s	remaining: 1.27s
345:	learn: 0.1319408	total: 4.28s	remaining: 1.26s
346:	learn: 0.1315830	total: 4.29s	remaining: 1.25s
347:	learn: 0.1312517	total: 4.3s	remaining: 1.24s
348:	learn: 0.1309423	total: 4.31s	remaining: 1.22s
349:	learn: 0.1305154	total: 4.33s	remaining: 1.21s
350:	learn: 0.1302718	total: 4.34s	remaining: 1.2s
351:	learn: 0.12

49:	learn: 0.5510513	total: 726ms	remaining: 5.78s
50:	learn: 0.5458415	total: 738ms	remaining: 5.74s
51:	learn: 0.5431698	total: 751ms	remaining: 5.71s
52:	learn: 0.5271107	total: 765ms	remaining: 5.7s
53:	learn: 0.5171090	total: 777ms	remaining: 5.67s
54:	learn: 0.5132078	total: 790ms	remaining: 5.65s
55:	learn: 0.5104744	total: 804ms	remaining: 5.63s
56:	learn: 0.5078676	total: 818ms	remaining: 5.61s
57:	learn: 0.5044519	total: 832ms	remaining: 5.59s
58:	learn: 0.4981432	total: 847ms	remaining: 5.59s
59:	learn: 0.4955782	total: 860ms	remaining: 5.56s
60:	learn: 0.4933858	total: 874ms	remaining: 5.55s
61:	learn: 0.4908643	total: 887ms	remaining: 5.52s
62:	learn: 0.4889550	total: 900ms	remaining: 5.5s
63:	learn: 0.4806356	total: 914ms	remaining: 5.48s
64:	learn: 0.4765668	total: 929ms	remaining: 5.47s
65:	learn: 0.4730518	total: 946ms	remaining: 5.47s
66:	learn: 0.4678054	total: 959ms	remaining: 5.45s
67:	learn: 0.4652860	total: 974ms	remaining: 5.44s
68:	learn: 0.4616832	total: 989ms

209:	learn: 0.2273952	total: 2.71s	remaining: 3.07s
210:	learn: 0.2268138	total: 2.72s	remaining: 3.06s
211:	learn: 0.2252956	total: 2.73s	remaining: 3.04s
212:	learn: 0.2246568	total: 2.75s	remaining: 3.03s
213:	learn: 0.2241113	total: 2.76s	remaining: 3.02s
214:	learn: 0.2232819	total: 2.77s	remaining: 3s
215:	learn: 0.2226843	total: 2.78s	remaining: 2.99s
216:	learn: 0.2220915	total: 2.8s	remaining: 2.98s
217:	learn: 0.2215297	total: 2.81s	remaining: 2.96s
218:	learn: 0.2209322	total: 2.82s	remaining: 2.95s
219:	learn: 0.2204814	total: 2.83s	remaining: 2.94s
220:	learn: 0.2199564	total: 2.85s	remaining: 2.92s
221:	learn: 0.2194364	total: 2.86s	remaining: 2.91s
222:	learn: 0.2176356	total: 2.87s	remaining: 2.9s
223:	learn: 0.2171179	total: 2.88s	remaining: 2.88s
224:	learn: 0.2163181	total: 2.89s	remaining: 2.87s
225:	learn: 0.2156322	total: 2.9s	remaining: 2.85s
226:	learn: 0.2151404	total: 2.92s	remaining: 2.84s
227:	learn: 0.2145066	total: 2.93s	remaining: 2.82s
228:	learn: 0.2138

...

321:	learn: 0.1416018	total: 3.8s	remaining: 1.49s
322:	learn: 0.1413423	total: 3.81s	remaining: 1.48s
323:	learn: 0.1408532	total: 3.82s	remaining: 1.46s
324:	learn: 0.1396598	total: 3.83s	remaining: 1.45s
325:	learn: 0.1389341	total: 3.85s	remaining: 1.44s
326:	learn: 0.1383532	total: 3.86s	remaining: 1.43s
327:	learn: 0.1381353	total: 3.87s	remaining: 1.42s
328:	learn: 0.1379190	total: 3.88s	remaining: 1.4s
329:	learn: 0.1377910	total: 3.89s	remaining: 1.39s
330:	learn: 0.1367914	total: 3.9s	remaining: 1.38s
331:	learn: 0.1362630	total: 3.92s	remaining: 1.37s
332:	learn: 0.1359554	total: 3.93s	remaining: 1.36s
333:	learn: 0.1356350	total: 3.94s	remaining: 1.34s
334:	learn: 0.1352867	total: 3.95s	remaining: 1.33s
335:	learn: 0.1349808	total: 3.96s	remaining: 1.32s
336:	learn: 0.1348164	total: 3.97s	remaining: 1.31s
337:	learn: 0.1340021	total: 3.98s	remaining: 1.3s
338:	learn: 0.1337859	total: 4s	remaining: 1.28s
339:	learn: 0.1335912	total: 4.01s	remaining: 1.27s
340:	learn: 0.13330

35:	learn: 0.6696009	total: 388ms	remaining: 4.44s
36:	learn: 0.6617608	total: 400ms	remaining: 4.44s
37:	learn: 0.6603165	total: 411ms	remaining: 4.43s
38:	learn: 0.6560416	total: 422ms	remaining: 4.42s
39:	learn: 0.6504229	total: 436ms	remaining: 4.45s
40:	learn: 0.6483113	total: 449ms	remaining: 4.46s
41:	learn: 0.6462356	total: 460ms	remaining: 4.44s
42:	learn: 0.6374883	total: 471ms	remaining: 4.43s
43:	learn: 0.6326224	total: 482ms	remaining: 4.43s
44:	learn: 0.6310530	total: 494ms	remaining: 4.42s
45:	learn: 0.6289871	total: 505ms	remaining: 4.41s
46:	learn: 0.6260021	total: 517ms	remaining: 4.41s
47:	learn: 0.6234796	total: 528ms	remaining: 4.4s
48:	learn: 0.6209461	total: 539ms	remaining: 4.39s
49:	learn: 0.6144024	total: 552ms	remaining: 4.39s
50:	learn: 0.6090376	total: 564ms	remaining: 4.39s
51:	learn: 0.6068216	total: 577ms	remaining: 4.39s
52:	learn: 0.6012712	total: 592ms	remaining: 4.41s
53:	learn: 0.5944180	total: 604ms	remaining: 4.41s
54:	learn: 0.5925494	total: 616m

206:	learn: 0.2716283	total: 2.37s	remaining: 2.76s
207:	learn: 0.2701384	total: 2.38s	remaining: 2.75s
208:	learn: 0.2688936	total: 2.4s	remaining: 2.74s
209:	learn: 0.2683056	total: 2.41s	remaining: 2.73s
210:	learn: 0.2674659	total: 2.42s	remaining: 2.72s
211:	learn: 0.2666862	total: 2.44s	remaining: 2.71s
212:	learn: 0.2642626	total: 2.45s	remaining: 2.7s
213:	learn: 0.2634464	total: 2.46s	remaining: 2.69s
214:	learn: 0.2620233	total: 2.47s	remaining: 2.68s
215:	learn: 0.2614384	total: 2.48s	remaining: 2.67s
216:	learn: 0.2608027	total: 2.5s	remaining: 2.66s
217:	learn: 0.2600886	total: 2.51s	remaining: 2.65s
218:	learn: 0.2592083	total: 2.52s	remaining: 2.63s
219:	learn: 0.2585381	total: 2.53s	remaining: 2.62s
220:	learn: 0.2579269	total: 2.54s	remaining: 2.61s
221:	learn: 0.2573330	total: 2.56s	remaining: 2.6s
222:	learn: 0.2553956	total: 2.57s	remaining: 2.59s
223:	learn: 0.2546509	total: 2.58s	remaining: 2.58s
224:	learn: 0.2540595	total: 2.59s	remaining: 2.57s
225:	learn: 0.25

381:	learn: 0.1538284	total: 4.35s	remaining: 751ms
382:	learn: 0.1535007	total: 4.36s	remaining: 740ms
383:	learn: 0.1528662	total: 4.37s	remaining: 728ms
384:	learn: 0.1524254	total: 4.38s	remaining: 717ms
385:	learn: 0.1522116	total: 4.39s	remaining: 706ms
386:	learn: 0.1514181	total: 4.4s	remaining: 694ms
387:	learn: 0.1510684	total: 4.41s	remaining: 682ms
388:	learn: 0.1507550	total: 4.42s	remaining: 671ms
389:	learn: 0.1501670	total: 4.43s	remaining: 659ms
390:	learn: 0.1498971	total: 4.44s	remaining: 647ms
391:	learn: 0.1497224	total: 4.45s	remaining: 636ms
392:	learn: 0.1494702	total: 4.46s	remaining: 624ms
393:	learn: 0.1491120	total: 4.47s	remaining: 612ms
394:	learn: 0.1488036	total: 4.48s	remaining: 601ms
395:	learn: 0.1485292	total: 4.49s	remaining: 589ms
396:	learn: 0.1480014	total: 4.5s	remaining: 578ms
397:	learn: 0.1477165	total: 4.5s	remaining: 566ms
398:	learn: 0.1474859	total: 4.51s	remaining: 554ms
399:	learn: 0.1468899	total: 4.52s	remaining: 543ms
400:	learn: 0.1

0.7064634146341463

In [37]:
from sklearn.ensemble import RandomForestClassifier

In [38]:
rc=RandomForestClassifier()

In [45]:
# accuracy using random forest
scores4= cross_val_score(rc, x, data_y, cv = 5, scoring = 'accuracy')
scores4.mean()

0.7637373737373737

In [46]:
lr=LogisticRegression()
lr.fit(x,data_y)

LogisticRegression()

In [47]:
# accuracy using logistic regression
scores3= cross_val_score(lr, x, data_y, cv = 5, scoring = 'accuracy')
scores3.mean()

0.804040404040404

Hyperparameter tuning using bayesian optimisation

In [134]:
!pip install bayesian-optimization

In [102]:
from sklearn.model_selection import cross_val_score
from bayes_opt import BayesianOptimization

In [103]:
def xgb_cv(n_estimators, max_depth, gamma, subsample, data, targets):
    estimator = XGBClassifier(
        n_estimators=n_estimators,
        max_depth = max_depth,
        gamma = gamma,
        # min_child_weight=min_child_weight,
        subsample = subsample,
        random_state = 2,
    )
    cval = cross_val_score(estimator, data, targets
                          , cv=5)
    return cval.mean()

In [104]:
def optimize_xgb(data, targets):
    def xgb_crossval(n_estimators, max_depth, gamma, subsample):
        return xgb_cv(
            n_estimators=int(n_estimators),
            max_depth = int(max_depth),
            gamma = gamma,
            # min_child_weight=min_child_weight,
            subsample=subsample,
            data=data,
            targets=targets,
        )

    optimizer = BayesianOptimization(
        f=xgb_crossval,
        pbounds={
            "n_estimators": (100, 500),
            "max_depth": (6,15),
            "gamma": (0,10),
            # "min_child_weight": (0,10),
            "subsample": (0.8,1.0)
        },
        random_state=1234,
        verbose=2
    )
    optimizer.maximize(n_iter=25, init_points=10)

    print("Final result:", optimizer.max)

In [105]:
print("--- Optimizing XGBoost ---")
optimize_xgb(x2, data_y)

--- Optimizing XGBoost ---
|   iter    |  target   |   gamma   | max_depth | n_esti... | subsample |
-------------------------------------------------------------------------
|  1        |  0.8308   |  1.915    |  11.6     |  275.1    |  0.9571   |
|  2        |  0.7595   |  7.8      |  8.453    |  210.6    |  0.9604   |
|  3        |  0.7464   |  9.581    |  13.88    |  243.1    |  0.9002   |
|  4        |  0.7506   |  6.835    |  12.41    |  248.1    |  0.9122   |
|  5        |  0.7817   |  5.031    |  6.124    |  409.1    |  0.9765   |
|  6        |  0.8174   |  3.649    |  11.54    |  130.2    |  0.8738   |
|  7        |  0.7374   |  9.331    |  11.86    |  258.9    |  0.9577   |
|  8        |  0.8041   |  3.168    |  11.11    |  447.7    |  0.8872   |
|  9        |  0.7684   |  8.021    |  7.294    |  381.7    |  0.9409   |
|  10       |  0.8308   |  2.188    |  14.32    |  276.9    |  0.9819   |
|  11       |  0.7593   |  0.0      |  6.0      |  286.0    |  0.8857   |
|  12      

In [145]:
def cb_cv(n_estimators, depth,data, targets):
    estimator = CatBoostClassifier(
        n_estimators=n_estimators,
#         learning_rate=learning_rate,
        depth=depth,
        random_state = 2,
        verbose = 0,
    )
    cval = cross_val_score(estimator, data, targets,
                            cv=5)
    return cval.mean()

In [146]:
def optimize_cb(data, targets):
    def cb_crossval(n_estimators, depth):
        return cb_cv(
            n_estimators=int(n_estimators),
#             learning_rate = learning_rate,
            depth = int(depth),
            data=data,
            targets=targets,
        )

    optimizer = BayesianOptimization(
        f=cb_crossval,
        pbounds={
            "n_estimators": (200, 600),
#             "learning_rate": (0.01,10),
            "depth": (4,16),
        },
        random_state=1234,
        verbose=2
    )
    optimizer.maximize(n_iter=10,init_points=5)

    print("Final result:", optimizer.max)


In [147]:
print("--- Optimizing Catboost ---")
optimize_cb(x, data_y)

--- Optimizing Catboost ---
|   iter    |  target   |   depth   | n_esti... |
-------------------------------------------------
|  1        |  0.7166   |  6.298    |  448.8    |
|  2        |  0.7166   |  9.253    |  514.1    |


KeyboardInterrupt: 

In [127]:
def lgb_cv(n_estimators, num_leaves, min_child_samples, subsample, data, targets):
    estimator = LGBMClassifier(
        n_estimators=n_estimators,
        num_leaves = num_leaves,
        min_child_samples=min_child_samples,
        subsample = subsample,
        random_state = 2
    )
    cval = cross_val_score(estimator, data, targets, cv=5)
    return cval.mean()

In [128]:
def optimize_lgb(data, targets):
    def lgb_crossval(n_estimators, num_leaves, min_child_samples, subsample):
        return lgb_cv(
            n_estimators=int(n_estimators),
            num_leaves = int(num_leaves),
            min_child_samples=int(min_child_samples),
            subsample=subsample,
            data=data,
            targets=targets,
        )

    optimizer = BayesianOptimization(
        f=lgb_crossval,
        pbounds={
            "n_estimators": (100,500),
            "num_leaves": (30,80),
            "min_child_samples": (5,30),
            "subsample": (0.6,1.0)
        },
        random_state=1234,
        verbose=2
    )
    optimizer.maximize(n_iter=25, init_points=20)

    print("Final result:", optimizer.max)

In [129]:
print("--- Optimizing Light GBM ---")
optimize_lgb(x2,data_y)

--- Optimizing Light GBM ---
|   iter    |  target   | min_ch... | n_esti... | num_le... | subsample |
-------------------------------------------------------------------------
|  1        |  0.7548   |  9.788    |  348.8    |  51.89    |  0.9141   |
|  2        |  0.7864   |  24.5     |  209.0    |  43.82    |  0.9207   |
|  3        |  0.7414   |  28.95    |  450.4    |  47.89    |  0.8004   |
|  4        |  0.746    |  22.09    |  385.1    |  48.51    |  0.8245   |
|  5        |  0.7595   |  17.58    |  105.5    |  68.64    |  0.9531   |
|  6        |  0.7503   |  14.12    |  346.2    |  33.77    |  0.7475   |
|  7        |  0.7594   |  28.33    |  360.6    |  49.86    |  0.9155   |
|  8        |  0.7637   |  12.92    |  327.2    |  73.46    |  0.7745   |
|  9        |  0.7729   |  25.05    |  157.5    |  65.21    |  0.8818   |
|  10       |  0.75     |  10.47    |  469.9    |  52.11    |  0.9637   |
|  11       |  0.7594   |  6.495    |  173.7    |  32.37    |  0.87     |
|  12    

In [40]:
from sklearn import svm

In [41]:
sv=svm.SVC()

In [48]:
# accuracy using SVM
scoresy= cross_val_score(sv, x, data_y, cv = 5, scoring = 'accuracy')
scoresy.mean()

0.7145454545454546

In [51]:
from sklearn.naive_bayes import MultinomialNB,GaussianNB,BernoulliNB

In [61]:
mn=MultinomialNB()

In [62]:
# accuracy using naive bayes
scores_n= cross_val_score(mn, x, data_y, cv = 5, scoring = 'accuracy')
scores_n.mean()

0.7771717171717172

In [57]:
x2=x.toarray()

In [71]:
from sklearn import tree

In [72]:
dc=tree.DecisionTreeClassifier()

In [73]:
# accuracy using Decision Tree
scores_d= cross_val_score(dc, x, data_y, cv = 5, scoring = 'accuracy')
scores_d.mean()

0.7231313131313131

In [80]:
from sklearn.linear_model import SGDClassifier

In [82]:
sgd=SGDClassifier(loss='modified_huber',shuffle=True,random_state=101)

In [83]:
# accuracy using stochastic gradient descent
scores_s= cross_val_score(sgd, x, data_y, cv = 5, scoring = 'accuracy')
scores_s.mean()

0.7773737373737374

In [84]:
from sklearn.neighbors import KNeighborsClassifier

In [85]:
knn=KNeighborsClassifier(n_neighbors=15)

In [86]:
# accuracy using KNN
scores_k= cross_val_score(knn, x, data_y, cv = 5, scoring = 'accuracy')
scores_k.mean()

0.7681818181818182

In [136]:
lg=LGBMClassifier(min_child_samples=int(11.974366158335911), n_estimators=int(364.3297920918007), num_leaves=int(74.41976662929886), subsample=0.7120571141255377)

In [137]:
# accuracy using LGBM
scores_l= cross_val_score(lg, x2, data_y, cv = 5, scoring = 'accuracy')
scores_l.mean()

0.795050505050505

In [112]:
Xg=XGBClassifier(gamma=1.9151945037889229,max_depth=int(11.598978939358487),n_estimators=int(275.0910956028458),subsample=0.9570717167427538)


In [113]:
# accuracy using XGboost
scores_x= cross_val_score(Xg, x2, data_y, cv = 5, scoring = 'accuracy')
scores_x.mean()

0.804141414141414